In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, plot_roc_curve, accuracy_score

pd.set_option('display.max_columns', None) 

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

%matplotlib inline
import matplotlib.pyplot as plt

from utils import XyScaler
from roc_curve2 import roc_curve

In [ ]:
event1 = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/Train/First_Health_Camp_Attended.csv')
event2 = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/Train/Second_Health_Camp_Attended.csv')
event3 = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/Train/Third_Health_Camp_Attended.csv')
patient_df = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/patient_attendance.csv')
patient_df1 = patient_df.copy()
print(patient_df1['Patient_ID'].nunique())


In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt 
plt.rcParams['figure.dpi'] = 200

In [ ]:
train = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/Train/Train.csv')
train1 = train.copy()
test_df = pd.read_csv('/home/allen/Galva/capstones/capstone2/data/test.csv') 
test_df1 = test_df.copy() 

In [ ]:
from numpy import argmax 

In [ ]:
patient_df1['Education_Score'].describe()

In [ ]:
# fig, ax = plt.subplots()
# counts = [848,1019,886,813,37633]
# count_labels = ['Online_Follower','Facebook_Shared','LinkedIn_Shared','Twitter_Shared', 'Total' ]

# ax.bar(range(len(counts)), counts)
# ax.set_xticks(range(len(counts)))
# ax.set_xticklabels(count_labels, rotation=45);
# plt.figsize(8,8)
# plt.savefig('Online_Sharing_Counts')

In [ ]:
#patient_df1[['Online_Follower','Facebook_Shared','LinkedIn_Shared','Twitter_Shared']].hist(figsize=(12,8))

In [ ]:
print(train.columns)
print(test_df.columns)
print(patient_df1.columns)

In [ ]:
p_id = patient_df1.Patient_ID.values
pp_id = test_df.Patient_ID.values

In [ ]:
def full_discolusure(pp_id,p_id):
    print(f'length of train is {len(p_id)} and length of patient is {len(pp_id)} ')
    p1,p2 = set(p_id) , set(pp_id)
    print(f' unique for train is {len(p1)} and unique for patient is {len(p2)} ')
    d = p1.difference(p2)
    print(f'The difference from p1 to p2 is {len(d)}')
    dd = p2.difference(p1)
    print(f'The difference from p2 to p1 is {len(dd)}')
    u, uu = p1.intersection(p2), p2.intersection(p1)
    return len(u),len(uu)
print(full_discolusure(pp_id,p_id))

In [ ]:
train1 = train1.merge(patient_df1, on=['Patient_ID'], how = 'left')

In [ ]:
test_df1 = test_df1.merge(patient_df1, on = ['Patient_ID'], how='left')

In [ ]:
train1[train1['Patient_ID'] == 489652 ]

In [ ]:
train1.shape

In [ ]:
test_df1.shape

In [ ]:
test_df1.columns

In [ ]:
train1.columns

In [ ]:
#train1 = train1.drop(['Unnamed: 0', 'Unnamed: 0.1','Registration_Date', 'Employer_Category','Patient_ID','First_Interaction', 'Health_Camp_ID'],axis=1)

In [ ]:
train1a = train1.drop(['Health_Camp_ID'],axis=1)

In [ ]:
test_df1 = test_df1.drop(['Unnamed: 0', 'Unnamed: 0.1','Registration_Date', 'Employer_Category','Patient_ID','First_Interaction','Health_Camp_ID'],axis=1)

In [ ]:
test_df1a = test_df1.drop(['Health_Camp_ID'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
y1, y2 = train1.pop('Event1_or_2') , test_df1.pop('Event1_or_2')

In [ ]:
X_train , X_test = train1 , test_df1

In [ ]:
X_train2 , X_test2 = train1a , test_df1a

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(solver="lbfgs")
logmodel.fit(X_train, y1)

In [ ]:
logmodel2 = LogisticRegression(solver="lbfgs")
logmodel2.fit(X_train2, y1)

In [ ]:
predictions = logmodel.predict_proba(X_test)[:,1]

In [ ]:
predictions_2 = logmodel2.predict_proba(X_test2)[:,1]

In [ ]:
predictions

In [ ]:
predict = logmodel.predict(X_test)

In [ ]:
preds, preds2 = predictions>.55 ,predictions>.52

In [ ]:
predsa, preds2a = predictions_2>.55 ,predictions_2>.52

In [ ]:
for name, coef in zip(preds2[1:], logmodel.coef_[0]):
    print("{0}: {1:0.4f}".format(name, coef))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y2,preds2) ) 

In [ ]:
print(classification_report(y2,predictions_2) )

In [ ]:
print(confusion_matrix(y2,preds2)) 

In [ ]:
print(classification_report(y2,preds) ) 

In [ ]:
from statsmodels.tools import add_constant
from statsmodels.discrete.discrete_model import Logit

In [ ]:
X = test_df1
X_const = add_constant(X, prepend=True)
y3 = y2.copy()
logit_model = Logit(y3, X_const).fit()

In [ ]:
logit_model.summary()

In [ ]:
kfold = KFold(n_splits=10)

accuracies = []
precisions = []
recalls = []

#X_train, X_test, y_train, y_test = train_test_split(X, y)

for train_index, test_index in kfold.split(X_train):
    model = LogisticRegression(solver="lbfgs")
    model.fit(X_train.iloc[train_index], y1.iloc[train_index])
    y_predict = model.predict(X_train.iloc[test_index])
    y_true = y1.iloc[test_index]
    accuracies.append(accuracy_score(y_true, y_predict))
    precisions.append(precision_score(y_true, y_predict))
    recalls.append(recall_score(y_true, y_predict))

print("Accuracy:", np.average(accuracies))
print("Precision:", np.average(precisions))
print("Recall:", np.average(recalls))

In [ ]:
''' 
n=20 
Accuracy: 0.6178722087906098
Precision: 0.6242284105951045
Recall: 0.9587480176361393
'''
#[[16088  2418]
 #[ 9349  7394]]
'''
              precision    recall  f1-score   support

           0       0.63      0.87      0.73     18506
           1       0.75      0.44      0.56     16743

    accuracy                           0.67     35249
   macro avg       0.69      0.66      0.64     35249
weighted avg       0.69      0.67      0.65     35249
'''

In [ ]:
def simple_plot(ax, x, y, x_label, y_label, title):
    ax.plot(x, y)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)

In [ ]:

model3 = LogisticRegression(solver='lbfgs')
model3.fit(X_train, y1)
probabilities2 = model3.predict_proba(X_test)[:, 1]
tpr, fpr, thresholds = roc_curve(probabilities2, y2)

fig, ax = plt.subplots(figsize=(12, 6))

simple_plot(ax, fpr, tpr, 
            "False Positive Rate (1 - Specificity)", 
            "True Positive Rate (Sensitivity, Recall)", 
            "ROC Plot of Patient Predictions OUT of BOX")

In [ ]:
'''
              precision    recall  f1-score   support

           0       0.63      0.87      0.73     18506
           1       0.75      0.44      0.56     16743

    accuracy                           0.67     35249
   macro avg       0.69      0.66      0.64     35249
weighted avg       0.69      0.67      0.65     35249
'''

In [ ]:
f_score = (2* .63 * .87)/(.63+.87)
print(f_score)

In [ ]:
'''
              precision    recall  f1-score   support

           0       0.73      0.35      0.48     18506
           1       0.54      0.85      0.67     16743

    accuracy                           0.59     35249
   macro avg       0.64      0.60      0.57     35249
weighted avg       0.64      0.59      0.57     35249


'''

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 200)
rfc.fit(X_train,y1)

In [ ]:
rfc_preds = rfc.predict(X_test)

In [ ]:
print(classification_report(y2,rfc_preds) ) 

In [ ]:
model4 = RandomForestClassifier(n_estimators = 200)
model4.fit(X_train, y1)
proba_Forest = model4.predict_proba(X_test)[:, 1]
tpr, fpr, thresholds = roc_curve(proba_Forest, y2)

fig, ax = plt.subplots(figsize=(12, 6))

simple_plot(ax, fpr, tpr, 
            "False Positive Rate (1 - Specificity)", 
            "True Positive Rate (Sensitivity, Recall)", 
            "ROC Plot Patient Predictions Random Forest")